# Perform EDA on raw data files

In [42]:
%cd /Volumes/External/source/GTPracticum/data 
%ls -salh

/Volumes/External/source/GTPracticum/data
total 13293816
      0 drwxr-xr-x   13 kariato  staff   416B Feb 11 14:17 ./
      0 drwxr-xr-x    7 kariato  staff   224B Feb 10 21:29 ../
     16 -rw-r--r--@   1 kariato  staff   6.0K Feb 10 14:58 .DS_Store
    464 -rw-r--r--@   1 kariato  staff   232K Feb 10 16:00 9deec473-6ab3-4c90-a2ec-86ae93b0a6e6.pdf
 733000 -rw-r--r--@   1 kariato  staff   358M Feb 10 14:56 archive (1).zip
2785400 -rw-rw-r--@   1 kariato  staff   1.3G Sep 19  2019 emails.csv
2109312 -rw-r--r--    1 kariato  staff   1.0G Feb 10 14:24 enron.sql
 427936 -rw-r--r--@   1 kariato  staff   209M Feb 10 14:24 enron.sql.gz
 865736 -rw-r--r--@   1 kariato  staff   423M Feb 10 14:35 enron_mail_20150507.tar.gz
6363112 -rw-r--r--    1 kariato  staff   3.0G Feb 11 14:17 enron_processed.csv
      0 drwxr-xr-x@  11 kariato  staff   352B Nov 10  2004 enron_with_categories/
   8840 -rw-r--r--@   1 kariato  staff   4.3M Feb 10 14:25 enron_with_categories.tar.gz
      0 drwxr-xr-x@ 152 kari

In [ ]:
!tar -xvf enron_mail_20150507.tar.gz

In [6]:
!gzip -dk enron.sql.gz

In [ ]:
!tar -xvf enron_with_categories.tar.gz

In [2]:
import pandas as pd
enron=pd.read_csv("emails.csv")

In [3]:
print(enron.shape)
print(enron.columns)

(517401, 2)
Index(['file', 'message'], dtype='object')


In [13]:
email_tags=['Subject','To','CC','BCC','SENT','From','Date','Content-Type','Message-ID','Mime-Version','Content-Transfer-Encoding',
            'X-From','X-To','X-cc','X-bcc','X-Folder','X-Origin','X-FileName']+['HTTP', '<HTTP', 'FAX', 'TIME', 'ALIAS', 'PHONE', '(SEE ATTACHED FILE', 'MKT_TYPE', 'TRANS_TYPE', 'PURPOSE', 'IMPACT', 'CONTACT(S)', 'OUTAGE', 'SC_ID', 'TRANS_DATE',
 'ENVIRONMENTS IMPACTED', 'BACKOUT', 'RECEIVED', 'START DATE', '10', 'SENT BY', 'DETAILS', '> SUBJECT', 'LOG MESSAGES', '<A HREF="HTTP', 'PARSING FILE -->> O',
 'ERROR', '11', 'TEL', '<< FILE', '> SENT', '12', '> TO', '09', '08', '> FROM', 'IMPORTANCE', 'E-MAIL', '04', '02', '03', '01', '05', 'EMAIL', 'RE', '07',
 '<MAILTO', 'ENGY_TYPE', 'INTERCHG_ID', 'TIE_POINT', '<IMG SRC="HTTP', 'LOCATION', 'PURCH_SALE', 'TRADING_SC', 'SCHED_TYPE', '[MAILTO', 'DEAL_NO', 'PNT_OF_INTRC',
 'X-MAILER', '>SUBJECT', '>TO', '> CC', 'PH', 'URL', 'CONTACT', '=09=09 SUBJECT', 'USA', 'MAILTO', '06', 'HREF="HTTP', '>FROM', '9', '', '=09=09 CC', 'HTTPS',
 'DESCRIPTION', 'WHEN', '1', 'GET YOUR FREE DOWNLOAD OF MSN EXPLORER AT HTTP', '3', 'LOAD_ID', '2', '>DATE', 'WHERE', 'INTERNET', 'SCHEDULED SYSTEM OUTAGES',
 'KAY', 'CELL', '=09=09 TO','9', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27','1','2','3','4','5','6','7','8','9','0',
 'CALENDAR ENTRY', 'COMPANY', 'SITARA', 'UNIFY', 'NT', 'TERMINAL SERVER', 'AZURIX', 'TELEPHONY', 'SUN/OSS SYSTEM', 'OS/2', 'FIELD SERVICES',
 'EB34 DATA CENTER - FACILITY OPERATIONS', 'ARDMORE DATA CENTER - FACILITY OPERATIONS', 'TITLE', 'TELEPHONE', "<TD STYLE='PADDING", '> HTTP',
 '<TR BGCOLOR=#EAEAEA ><TD CLASS=TD2 ><A HREF="HTTP', 'RETURN-PATH', '<TR BGCOLOR=#F0F0F0 ><TD CLASS=TD2 ><A HREF="HTTP', 'OTHER SYSTEMS', 'EDI SERVER', 'REQUEST ID',
 'REQUEST CREATE DATE', 'RESOURCE NAME', 'RESOURCE TYPE', 'ACCESS ONLINE AT', 'MESSAGING', 'RATE', '18','SPECIFIC HELP', '*TO', 'THE REPORT NAMED', 'VINCE',
 '7', 'SOURCE', 'AIRCRAFT', 'MARK', 'INDIA', 'PLEASE NOTE', 'ACTIVITY', '17', "STYLE='FONT-SIZE", '=09=09HTTP', 'UNIT_ID', '<FONT  FACE="TIMES NEW ROMAN"><B><A HREF="HTTP',
 '4', 'OFFICE', 'ENRON CENTER SOUTH DATA CENTER - FACILITY OPERATIONS', '5', 'A', 'PASSWORD', '=09FROM', '15', '6',
 "<P CLASS=MSONORMAL ALIGN=RIGHT STYLE='TEXT-ALIGN", 'PAGER', 'FACSIMILE', 'EMPLOYEE NAME', '> > TO', 'BODY', 'EVENT', 'HEADLINE', 'X-MS-TNEF-CORRELATOR', 'X-MS-MAIL-PRIORITY',
 'LENGTH', 'ACCESS STORIES ONLINE AT', 'TANA', '14', '> > SENT', 'LOAD-DATE', 'GET YOUR PRIVATE, FREE E-MAIL FROM MSN HOTMAIL AT HTTP',
 'JEFF', 'CONTENT-DISPOSITION', 'SECTION', 'REPORT TOTAL', 'EXPENSE REPORT NAME', 'AMOUNT DUE EMPLOYEE', 'SRC="HTTP', '19', 'EFFECTIVE DATE',
 '<< OLE OBJECT', 'AIRPORT', 'STEVE', '>> > > TO', 'BYLINE', '> > FROM', '(HTTP', 'SPONSORED BY', 'X-VERSION', '<TD><IMG SRC="HTTP', 'NAME', 'ADDRESS', '> FAX',
 '>CC', '>HTTP', 'BUY.COM PRICE', 'REFERENCE NUMBER',  'PLACE', '$21 A DAY IN', 'E-MAIL CATEGORY',
 'TOPIC', 'VOICE', 'ALL', 'JOHN', '$18 A DAY IN', '> (SEE ATTACHED FILE', 'AUTHOR', 'WEB', '>SENT', 'USER ID', 'TERM', 'PARTICIPANT CODE', 'MOBILE',
 'UK', 'X-MIMEOLE', 'GREETINGS', 'TO UNSUBSCRIBE FROM THIS GROUP, SEND AN EMAIL TO', 'VISIT HTTP', "UNRECOGNIZED DATABASE FORMAT 'R", 'FILE',
  'THE FOLLOWING EXPENSE REPORT IS READY FOR APPROVAL', 'REGULAR', 'O', '16', 'ACCESS ALL ONLINE PRESENTATIONS AT',
 'TERMS OF USE', 'COLOR', 'NOTICE', 'EES', 'X-PRIORITY',
 '=0F', 'FTP', 'SUSAN', 'STATUS', 'DEGREE', '$26 A DAY IN', '> PHONE', '(EMBEDDED IMAGE MOVED TO FILE', 'FONT-FAMILY', 'HOME', 'CLICK HERE', 'CRUDE     HTTP',
 'NATURAL GAS     HTTP', 'TO FILE', 'HOUSTON', 'SENSITIVITY', '=09 BROWN', 'DAVID', 'USUAL EVENT TIME', 'APPROXIMATE TIME OF MESSAGE AVAILABILITY',
 'STANDARD EVENT MESSAGE', 'FONT-SIZE',  '> >SUBJECT', 'RICK', 'JIM', 'PHOTO', '<A HREF=3D"HTTP', 'X-MSMAIL-PRIORITY', 'YOUR USE OF YAHOO! GROUPS IS SUBJECT TO HTTP',
 'PASSCODE', '<OMNICREATEDDATE>5/9/2000 5', 'TEL.', 'DOCUMENT.WRITE(\'<IMG SRC="HTTP', 'DATELINE', 'ENRON', 'WEB SITE', 'X-MIMETRACK', 'BRAZIL', '$23 A DAY IN', 'NOTICE ID', 'DISTILLATE     HTTP', 'UNLEADED     HTTP', '> > CC', '> >TO', 'ATTN', '>    SUBJECT',
 'AT HTTP', '> DATE', '=09 Q', 'PRONUNCIATION', 'CALL SOS MEDICAL EMERGENCY', 'ENRON ENTITY', 'RICHARD', '>    SENT', 'FEEDBACK DUE DATE', "TODAY'S FOCUS", '$20 A DAY IN', 'WEBSITE', '> >FROM', '=09HTTP', 'MSGID',
 'PRESS RELEASE', 'PRODUCT LINE', 'DAVE', 'RESERVATION CONFIRMED         2', 'QUESTION', 'STAR WARS', '=09SENT BY', 'CONFIRMATION', 'PHONE NO']
email_tags=[i.upper() for i in email_tags]

In [ ]:
email_lines_top={}

In [36]:
from collections import defaultdict
import re
tags=defaultdict(int)
email_lines=defaultdict(int)
email_list=[]
email_pattern = re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b')
phone_pattern = re.compile(r'\b\d{3}[-.]?\d{3}[-.]?\d{4}\b')
for email_header,email_row in enron.iterrows():
    email_txt=email_row["message"]
    body=[]
    bodyShort=[]
    email_header={}
    for email_line in email_txt.splitlines():
        header_parts=email_line.split(":")
        if len(header_parts)>1:
            tag=header_parts[0].upper().strip()
            if  tag in email_tags:
                if tag in email_header:
                    email_header[tag]+='>> '+':'.join(header_parts[1:-1])
                else: 
                    email_header[tag]=':'.join(header_parts[1:-1])
                if header_parts[0]=='Subject':
                    body.append(email_line)
                    bodyShort.append(email_line)
            else:
                if '/Enron@ENRON,' not in email_line and '@enron.com' not in email_line:
                    body.append(email_line)
                    if email_line not in email_lines_top:
                        bodyShort.append(email_line)                
                    tags[tag]+=1
        else:
            if '/Enron@ENRON,' not in email_line and '@enron.com' not in email_line and email_pattern.search(email_line) is None and phone_pattern.search(email_line) is None:
                body.append(email_line)
                email_lines[email_line]+=1
                if email_line not in email_lines_top:
                    bodyShort.append(email_line)

    email_header['body']='\n'.join(body)
    email_header['body_short']='\n'.join(bodyShort)
    email_header['raw']=email_txt
    email_list.append(email_header)
tagsEdited={i:j for i,j in tags.items() if j>40}
email_df=pd.DataFrame(email_list)
bad_tag=pd.DataFrame([tagsEdited]).T.sort_values(0,ascending=False)


In [26]:
email_lines_top={i:j for i,j in email_lines.items() if j>100}
print(len(email_lines_top))


5242


In [ ]:
bad_tag.reset_index()['index'].tolist()

In [27]:
print(email_lines_top[''])


5923072


In [37]:
email_df2=email_df.sample(1000)
print(email_df2['body'].tail(1).to_list()[0])

Subject: Cost Center Number

Mark, your Cost Center number is 105657.  Deb


Deb Korkmas
Enron North America Corp.
ENA Legal Dept.
1400 Smith Street, Room 3819
Houston, TX  77002


In [38]:
print(email_df2['body_short'].tail(1).to_list()[0])

Subject: Cost Center Number
Mark, your Cost Center number is 105657.  Deb
ENA Legal Dept.


In [39]:
print(email_df2['raw'].tail(1).to_list()[0])

Message-ID: <28325331.1075860180773.JavaMail.evans@thyme>
Date: Wed, 3 Jan 2001 09:15:00 -0800 (PST)
From: deb.korkmas@enron.com
To: mark.taylor@enron.com
Subject: Cost Center Number
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Deb Korkmas
X-To: Mark Taylor
X-cc: 
X-bcc: 
X-Folder: \Mark_Taylor_Jun2001\Notes Folders\All documents
X-Origin: Taylor-M
X-FileName: mtaylor.nsf

Mark, your Cost Center number is 105657.  Deb


Deb Korkmas
Enron North America Corp.
ENA Legal Dept.
1400 Smith Street, Room 3819
Houston, TX  77002
Phone:  713-853-5448
Fax:  713-646-3393


In [41]:
email_df.to_csv("enron_processed.csv")